In [172]:
from selenium import webdriver
from selenium.webdriver import ActionChains
import  time

# Ubicacion del driver e inicializacion del driver
chromedriver = "C:/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')

# Inicializar actionChains que controla las interacciones como el mouse
actionChains = ActionChains(driver)

# Indicar que pagina se desea abrir
driver.get('https://www.juegosinfantilespum.com/laberintos-online/12-auto-buhos.php')

# Esperar a que cargue la pagina y buscar el canvas donde se encuentra el juego
time.sleep(5)
canvas = driver.find_element_by_id("canvas")

# Mover el mouse a la posicion del canvas y dar click, esto es suficiente para iniciar el juego
actionChains.move_to_element(canvas).click(canvas).perform()
time.sleep(1)

# Guardar las los objetos del juego en objts. la funcion execute_script ejecuta un script de
# JavaScript como si fuera la consola de chrome.
# El objeto que contiene toda la informacion del nivel en el juego es exportRoot, el problema
# es que no se puede utilizar directamente ya que es recursivo e infinito, existen librerias y 
# algoritmos para solucionar eso, pero ya que solo es necesario recoger algunos datos de todo
# el objeto, se me hace mas sencillo con un loop.
# Este script retorna el diccionario res y se traduce automaticamente a un diccionario en python.
objts = (driver.execute_script('''
	let rs = exportRoot.children;
	let res = {
		'obst' : []
	};

	for(let i = 1; i < rs.length; i++){
		console.log(rs[i].name);
		if(rs[i].name == 'personaje'){
			res['player'] = [rs[i].x, rs[i].y];
		}
        if(rs[i].name == 'puerta'){
			res['finish'] = [rs[i].x, rs[i].y];
		}
		else if(rs[i].name != null && rs[i].name.charAt(0) == "o"){
 			res['obst'].push([rs[i].x, rs[i].y]);
		}
		else if(rs[i].name == 'pared'){
 			res['wall'] = rs[i].children[0].graphics._instructions
		}
	}

	return res;
'''))

In [173]:
# En el juego las paredes estan dentro de walls, estas estan dibujadas en varias partes
# siguiendo el orden:
# Paredes exteriores (el cuadro del canvas) -> paredes exteriores del laberinto -> muros dentro del laberinto.
# Conociendo este orden, a las paredes exteriores del laberinto se les asigna el color blanco para que posteriormente
# se pueda distinguir el camino que se puede tomar del que no, esto se guarda en prc_walls.
# temp_cdr es una variable auxiliar para guardar las coordenadas que forma algun grupo de paredes.

prc_walls = []
i = -1
temp_cdr = []
for pr in objts['wall']:
    if 'bitmap' not in pr.keys():
        if 'x' in pr.keys() and 'y' in pr.keys():
            temp_cdr.append([pr['x'], pr['y']])
        else:
            if i >= 0:
                # Aca se guardan las coordenadas que forman las paredes y el color asignado.
                prc_walls.append({'coords': temp_cdr, 'color': i == 1})
                temp_cdr = []
            i += 1
objts['wall'] = prc_walls

In [174]:
# Guardar el diccionario como json en ./data/data.json
import json
with open('data/data.json', 'w') as fp:
    json.dump(objts, fp)

In [175]:
# Ejecuta un sketch en processing que dibuja el mapa almacenado data.json y crea una imagen del mismo
# a blanco y negro en ./maze.jpg. (Esto puede hacerse en python directamente, pero estoy mas familiarizado
# con processing para este tipo de cosas).

import os
os.system("processing-java --sketch=%cd%/proc_img --run")

0

In [183]:
# Abre la imagen creada previamente con la libreria PIL y la convierte en una matriz n*m de 1 y 0. Esto es
# debido a que PIL crea una matriz n*m*3 donde la ultima dimension son los 3 canales de color rgb.

from PIL import Image, ImageOps
import numpy as np

im = Image.open('maze.png')

# print(len(pix))
# pix = pix.T[0].T
# im.thumbnail(im.size/38,Image.ANTIALIAS)

gray = im.convert('L')
bw = np.asarray(gray).copy()
bw[bw < 128] = 0 
bw[bw >= 128] = 255 
im = Image.fromarray(bw)
im = im.convert("RGBA")

pixdata = im.load()

width, height = im.size
for y in range(height):
    for x in range(width):
        if pixdata[x, y] == (255, 255, 255, 255):
            pixdata[x, y] = (255, 255, 255, 0)

pix = np.array(im)

fn = np.zeros(shape=(len(pix),len(pix[0])))
print(len(fn),len(fn[0]))

for i, x in enumerate(pix):
    for j, y in enumerate(x):
        fn[i][j] = 1 if np.sum(y) > 255*3/2 else 0

unique, count = np.unique(fn, return_counts = True) 
print(unique, count)

367 730
[0. 1.] [ 81754 186156]


In [177]:
def mp(value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))

In [184]:
# En la matriz creada previamente, se mete al jugador como un 2 y a la casa como un 3.

objetos = fn
coords_plyr = objts['player']
# print(coords_plyr)
coords_fnsh = objts['finish']
objetos[int(coords_plyr[1])][int(coords_plyr[0])] = 2
objetos[int(coords_fnsh[1])][int(coords_fnsh[0])] = 3

# print(objetos, int(mp(coords_fnsh[1], 0, 367, 0, 10)))

# Finalmente se guarda la matriz en ./data/out.txt
np.savetxt("data/out.txt", objetos, fmt="%d")

In [36]:
import numpy as np
from math import pi, sin, cos




'a' * 30

90.0 180.0 270.0 270.0 0.0 0.0


'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'